In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

S x S grid

predicitons = S * S * ( B * 5 + C )

24 convs + 2 fc

pretrain conv on ImageNet

input image = 448 * 448

output = 7 * 7 * 30

In [ ]:
class ConvLayers(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=192, kernel_size=7, stride=2, padding=3)  
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv2d(in_channels=192, out_channels=256, kernel_size=3, padding=1)  
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=128, kernel_size=1),
            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1),  
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)  
        )
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv3_5 = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(in_channels=512, out_channels=256, kernel_size=1),
                nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, padding=1)  
            ) for _ in range(4)]
        )
        
        self.conv4 = nn.Sequential(
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=1),
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)  
        )
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.conv4_5 = nn.Sequential(
            *[nn.Sequential(
                nn.Conv2d(in_channels=1024, out_channels=512, kernel_size=1),
                nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=3, padding=1)  
            ) for _ in range(2)]
        )
        
        self.conv5 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),  
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, stride=2, padding=1)  
        )
        
        self.conv6 = nn.Sequential(
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1),  
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=3, padding=1)  
        )
    
    def forward(self, x):
        x = self.maxpool1(F.relu(self.conv1(x)))
        print("Shape after conv1 and maxpool1:", x.shape)
        
        x = self.maxpool2(F.relu(self.conv2(x)))
        print("Shape after conv2 and maxpool2:", x.shape)
        
        x = self.maxpool3(F.relu(self.conv3(x)))
        print("Shape after conv3 and maxpool3:", x.shape)
        
        x = self.maxpool4(F.relu(self.conv4(x)))
        print("Shape after conv4 and maxpool4:", x.shape)
        
        x = F.relu(self.conv5(x))
        print("Shape after conv5:", x.shape)
        
        x = F.relu(self.conv6(x))
        print("Shape after conv6:", x.shape)
        
        return x

In [9]:
class FCLayers(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc1 = nn.Linear(7*7*1024, 4096)
        self.fc2 = nn.Linear(4096, 7*7*30)
        
    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = F.gelu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
conv = ConvLayers()
fc = FCLayers()

x = torch.randn(1, 3, 448, 448)
x = conv(x)
print(x.shape)
x = fc(x)
print(x.shape)

Shape after conv1 and maxpool1: torch.Size([1, 192, 112, 112])
Shape after conv2 and maxpool2: torch.Size([1, 256, 56, 56])
Shape after conv3 and maxpool3: torch.Size([1, 512, 28, 28])
Shape after conv4 and maxpool4: torch.Size([1, 1024, 14, 14])
Shape after conv5: torch.Size([1, 1024, 7, 7])
Shape after conv6: torch.Size([1, 1024, 7, 7])
torch.Size([1, 1024, 7, 7])
torch.Size([1, 1470])
